In [7]:
import or_gym
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

from stable_baselines3 import SAC
from stable_baselines3.sac.policies import MlpPolicy as SACPolicy

from stable_baselines3 import A2C
from stable_baselines3.a2c.policies import MlpPolicy as A2CPolicy

from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy as PPOPolicy

from sb3_contrib import ARS
from sb3_contrib.ars.policies import ARSPolicy

from sb3_contrib import RecurrentPPO
from sb3_contrib.ppo_recurrent.policies import RecurrentActorCriticPolicy

from sb3_contrib import TQC
from sb3_contrib.tqc.policies import MlpPolicy as TQCPolicy

from sb3_contrib import TRPO
from sb3_contrib.trpo.policies import MlpPolicy as TRPOPolicy

In [8]:
def eval_model_on_env(model, env, n_eval_episodes):
    rewards = np.empty(shape=(n_eval_episodes, env.periods))
    obs = env.reset()

    for episode in range(n_eval_episodes):
        for timestep in range(env.periods):
            action = model.predict(obs)
            obs, reward, _, _ = env.step(action[0])
            rewards[episode,timestep] = reward 

        obs = env.reset()

    return rewards


def eval_best_model(env_name, algorithm, name, n_eval_episodes):
    env = or_gym.make(env_name)
    model = algorithm.load(f'./data/{env_name}/{algorithm.__name__}/{name}/best_model', env=env)
    rewards = eval_model_on_env(model, env, n_eval_episodes)

    np.save(f'./data/{env_name}/{algorithm.__name__}/{name}/best_model_eval_rewards.npy', rewards) 



In [9]:
eval_best_model('InvManagement-v1', PPO, 'default', 100)

In [ ]:
eval_best_model('InvManagement-v1', A2C, 'default', 100)